In [72]:
import pandas as pd
import numpy as np
import re
from shapely.geometry import Point, Polygon
import geopandas as gpd
import fiona


### Transform Data to GeoData

In [73]:
def to_float(i):
    """
    el parámetro i es un del tipo str
    -----------------------------------
    La función to_float convierte a 'i' a tipo flotante
    -----------------------------------
    devuelve un valor del tipo float
    
    """
    i = float(i)
    return i

In [74]:
def transform(row):
    """
    el parámetro row es un valor del tipo str
    ----------------------------------------------
    separa los elementos de cada row de acuerdo a ciertos caracteres
    depura aquellos que no cumplen ciertas condiciones asignandoles el valor de lista vacia
    transforma cada cadena de texto a valores del tipo float
    ------------------------------------------------
    Retorna coor la cual puede ser una lista con valores flotantes 
    o una lista vacía.

    """
    input = row.split(',')

    size_list = len(input)


    if size_list ==1:
        coor = []
    elif len(input) ==2:
        try:
            pre_punto = map(to_float,input)
        except ValueError as e:
            print(e)
            coor=list(pre_punto)
    elif len(input) >2:
        not_point = re.split(r",0\\n\\n|,0|0\s-|,",row)
        if ("" in not_point):
            not_point.remove("")
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
        else:
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
        return coor


In [75]:
def to_matrix(listado):
    """
    el parámetro listado es un objeto del tipo lista
    -----------------------------------
    transforma una lista a un array de numpy
    asigna la forma del array
    -----------------------------------
    devuelve una matriz
    """
    n = len(listado)
    c = 2
    f =  n/c
    f = int(f)
    vector = np.array(listado)
    matriz = vector.reshape(f,c)
    return matriz

In [76]:
def arreglo(listado):

    ar = listado.sort()
    return ar

In [77]:
def geometry_apply(x):
    """
    el parámetro x es una lista
    ------------------------------------------
    de acuerdo a la longitud de la lista aplica una o dos funciones
    -----------------------------------------
    devuelve geometry que puede ser un objeto Point, Polygon o un valor None
    """

    n = x

    if len(n) == 2:
            orden=arreglo(n)
            geometry = Point(orden)
    elif len(n) >5 and len(n) %2 == 0: #Nueva validación
        listado = to_matrix(n)
        geometry = Polygon(listado)
    else:
        geometry = None
    return geometry


In [78]:
def typeshape(x):
    """
    El parámetro x es un str
    ----------------------------------------
    De acuerdo al valor genera ciertas cadenas de texto
    ---------------------------------------
    retorna label que es un str
    """
    n=x
    if int(n) == 2:
        label = "Point"
    else:
        label = "Polygon"
    return label    

In [79]:
PATH = input('Please enter the path where are you working')

In [80]:
PATH = PATH + '\\'

In [81]:
PATH

'C:\\Users\\JMELENDEZ\\OneDrive - CIMMYT\\Desktop\\Aguas Firmes Calera\\AguasFirmes\\'

In [82]:
target_file = 'bitacoras_pv22.csv'

In [92]:
puntos = PATH+'points.geojson'
poligonos = PATH+'polygons.geojson'

In [83]:
def main(file_input):
    """
    file_input recibe al archivo Sitios.txt
    -----------------------------------------------
    Lee el archivo
    genera columnas y aplica funciones
    crea 3 subsets distintos
    elimina columnas temporales y columna Coordenadas areas
    exportar los sets a un formato específico
    -----------------------------------------------
    genera los archivos de salida o un error
    """
    try:
        sitios= pd.read_csv(file_input)
        sitiosbd = sitios.copy()
        sitiosbd['info_geo'] = sitiosbd['Coordenadas']
        sitiosbd['info_geo']=sitiosbd['info_geo'].astype(str)
        sitiosbd['info_geo']= sitiosbd['info_geo'].apply(transform)
        desechados=sitiosbd[sitiosbd['info_geo'].isnull()]
        sitiosbd_depurate=sitiosbd[sitiosbd['info_geo'].notna()]
        sitiosbd_depurate['n_vertices'] = sitiosbd_depurate['info_geo'].apply(lambda x:len(x))
        sitiosbd_depurate['type_geometry'] = sitiosbd_depurate['info_geo'].apply(geometry_apply)
        sitiosbd_depurate=sitiosbd_depurate[sitiosbd_depurate['type_geometry'].notna()]
        desechados_2=sitiosbd_depurate[sitiosbd_depurate['type_geometry'].isnull()]
        desechados = pd.concat([desechados_2,desechados],axis=1)
        desechados.to_csv(f'{PATH}errors.csv')
        sitiosbd_depurate['type_shape'] = sitiosbd_depurate['n_vertices'].apply(typeshape)
        sitiosbd_polygon = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Polygon']
        sitiosbd_point = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Point']
        gdf_points_sitios = gpd.GeoDataFrame(sitiosbd_point,geometry='type_geometry',crs='CRS84')
        gdf_polygons_sitios = gpd.GeoDataFrame(sitiosbd_polygon,geometry='type_geometry',crs='CRS84')
 
        eliminar = ['Coordenadas','info_geo','n_vertices','type_shape','Nombre Productor','Bitacora registrada por']
        gdf_points_sitios=gdf_points_sitios.drop(columns=eliminar)
        gdf_polygons_sitios=gdf_polygons_sitios.drop(columns=eliminar)

        gdf_points_sitios.to_file(puntos,crs='CRS84', driver='GeoJSON')
        gdf_polygons_sitios.to_file(poligonos,crs='CRS84', driver='GeoJSON')
    except FileNotFoundError as e:
        print(f'Por favor ejecute el script en el mismo directorio donde se encuentre el archivo Sitio.txt {e}')

In [84]:
if __name__ == "__main__":
    main(f'{PATH}{target_file}')

c:\Users\JMELENDEZ\AppData\Local\Programs\Python\Python38\lib\site-packages\geopandas\io\file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


### Join Oldset with new one

In [85]:
last_geojson_file = PATH+'layer_poligonos.geojson'

In [86]:
last_geodf = gpd.read_file(last_geojson_file)

In [88]:
last_geodf.head(2)

,ID,Tipo de bitacora,Tipo de superficie,ID parcela,ID del sitio,Nombre de la Parcela,Estado,Municipio,Año,Ciclo,Regimen hidrico,Superficie sembrada,Superficie Total,Cultivo,Labranza,Tipo de semilla,Variedad,geometry
0,BitAg-2022-0626-0141-9mNZj-BHEWv,AE,Innovacion,RParc-2022-0626-0139-VWc1d-glJO4,Sitio-2022-0524-1107-aYZTC-h2hch,ALFREDO DOMINGUEZ1,Zacatecas,Panuco,OI_2021_2022,OI,RIEGO,2.0,2.0,AJO,Labranza convencional,Otro Criolla,CRIOLLO,"MULTIPOLYGON (((-102.58961 22.98375, -102.5887..."
1,BitAg-2022-0525-0312-tFiZR-NONKi,AE,Innovacion,RParc-2022-0518-1025-KuiWV-pG8zK,Sitio-2022-0413-1202-gykMQ-coVuZ,AJO,Zacatecas,Calera,OI_2021_2022,OI,RIEGO,4.5,4.5,Ajo,Labranza minima reducida,Otro Criolla,AJO,"MULTIPOLYGON (((-102.65673 22.93083, -102.6555..."


In [89]:
last_geodf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   ID                    162 non-null    object  
 1   Tipo de bitacora      162 non-null    object  
 2   Tipo de superficie    162 non-null    object  
 3   ID parcela            162 non-null    object  
 4   ID del sitio          162 non-null    object  
 5   Nombre de la Parcela  162 non-null    object  
 6   Estado                162 non-null    object  
 7   Municipio             162 non-null    object  
 8   Año                   162 non-null    object  
 9   Ciclo                 162 non-null    object  
 10  Regimen hidrico       162 non-null    object  
 11  Superficie sembrada   162 non-null    float64 
 12  Superficie Total      162 non-null    float64 
 13  Cultivo               162 non-null    object  
 14  Labranza              162 non-null    object  
 15

In [93]:
#read the new  one 
new_geodf = gpd.read_file(poligonos)

In [94]:
new_geodf.head(2)

,ID de bitacora,Tipo de bitacora,Tipo de superficie,ID del productor,ID parcela,ID del sitio,Nombre de la Parcela,Estado,Municipio,coordenadas duplicadas,...,Rendimiento(ton/ha),Precio de venta(MXN/ton),UN (kg/ha),Uso de N (UN kg/ton),Tipo de riego,No Riego,Consumo de agua (m3/ha),Intensidad de agua (m3/ton),Productividad de agua (kg/m3),geometry
0,BitAg-2022-0627-0546-ZWlCo-3aPv1,AE,Innovacion,RProd-2022-0616-0548-spxku-QJZx9,RParc-2022-0627-0532-dtDy0-wPQjX,Sitio-2022-0712-1234-Rs8vG-7D67w,LA ESTACION,Zacatecas,Fresnillo,NaN,...,1.8,17500.0,37.000,20.555556,Riego_por_goteo,3.0,1260.0,700.000000,1.428571,"POLYGON ((-102.78167 23.20027, -102.78131 23.2..."
1,BitAg-2022-0812-0534-vFQh5-YrZIO,AE,Innovacion,RProd-2022-0616-0651-dfUDO-trYu7,RParc-2022-0812-0530-fYMc9-ZWYFv,Sitio-2022-0719-0922-EpjMU-qIaqn,NELSON ALANIS GARCIA,Zacatecas,Fresnillo,NaN,...,3.0,90000.0,31.825,10.608333,Riego_por_goteo,34.0,4820.0,1606.666667,0.622407,"POLYGON ((-102.74475 23.18956, -102.74476 23.1..."


In [103]:
new_geodf['Semilla']

0       Frijol_Hibrida
1        Chile_Criolla
2         Maiz_Hibrida
3         Maiz_Hibrida
4         Maiz_Hibrida
            ...       
286      Chile_Criolla
287    Frijol_Mejorada
288       Maiz_Hibrida
289      Chile_Hibrida
290      Chile_Criolla
Name: Semilla, Length: 291, dtype: object

In [95]:
new_geodf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   ID de bitacora                 291 non-null    object  
 1   Tipo de bitacora               291 non-null    object  
 2   Tipo de superficie             291 non-null    object  
 3   ID del productor               291 non-null    object  
 4   ID parcela                     291 non-null    object  
 5   ID del sitio                   291 non-null    object  
 6   Nombre de la Parcela           291 non-null    object  
 7   Estado                         291 non-null    object  
 8   Municipio                      291 non-null    object  
 9   coordenadas  duplicadas        6 non-null      float64 
 10  Anio Bitacora                  291 non-null    object  
 11  Ciclo                          291 non-null    object  
 12  Regimen hidrico             

In [99]:
last_geodf.columns

Index(['ID', 'Tipo de bitacora', 'Tipo de superficie', 'ID parcela',
       'ID del sitio', 'Nombre de la Parcela', 'Estado', 'Municipio', 'Año',
       'Ciclo', 'Regimen hidrico', 'Superficie sembrada', 'Superficie Total',
       'Cultivo', 'Labranza', 'Tipo de semilla', 'Variedad', 'geometry'],
      dtype='object')

In [110]:
last_geodf.rename(columns={'Año':'Anio Bitacora'},inplace=True)

In [105]:
new_geodf.columns

Index(['ID de bitacora', 'Tipo de bitacora', 'Tipo de superficie',
       'ID del productor', 'ID parcela', 'ID del sitio',
       'Nombre de la Parcela', 'Estado ', 'Municipio ',
       'coordenadas  duplicadas', 'Anio Bitacora', 'Ciclo', 'Regimen hidrico',
       'Superficie sembrada', 'Superficie Total', 'Cultivo ', 'tipo  labranza',
       'tecnologia', 'Realiza  AC', 'Tecnificacion  Riego', 'Semilla',
       'Producto', 'Variedad', 'Rendimiento(ton/ha)',
       'Precio de venta(MXN/ton)', 'UN (kg/ha)', 'Uso de N (UN kg/ton)',
       'Tipo de riego', 'No Riego', 'Consumo de agua (m3/ha)',
       'Intensidad de agua (m3/ton)', 'Productividad de agua (kg/m3)',
       'geometry'],
      dtype='object')

In [109]:
new_geodf.rename(columns={'ID de bitacora':'ID','Estado ':'Estado','Municipio ':'Municipio','tipo  labranza':'Labranza','Semilla':'Tipo de semilla',
                          'Cultivo':'Cultivo'},inplace=True)

In [113]:
new_geodf.rename(columns={'Cultivo ':'Cultivo'},inplace=True)

In [133]:
lista_cols = list(last_geodf.columns)

In [134]:
new_geodf = new_geodf[lista_cols]

In [135]:
last_set = PATH +'last_set.geojson'

In [144]:
##Just to prevent errror about projection, it is necessary that both files are same projection
last_geodf.to_file(last_set,crs='CRS84', driver='GeoJSON')

In [145]:
same_pro_last_set = gpd.read_file(last_set)

In [146]:
same_pro_last_set.head(2)

,ID,Tipo de bitacora,Tipo de superficie,ID parcela,ID del sitio,Nombre de la Parcela,Estado,Municipio,Anio Bitacora,Ciclo,Regimen hidrico,Superficie sembrada,Superficie Total,Cultivo,Labranza,Tipo de semilla,Variedad,geometry
0,BitAg-2022-0626-0141-9mNZj-BHEWv,AE,Innovacion,RParc-2022-0626-0139-VWc1d-glJO4,Sitio-2022-0524-1107-aYZTC-h2hch,ALFREDO DOMINGUEZ1,Zacatecas,Panuco,OI_2021_2022,OI,RIEGO,2.0,2.0,AJO,Labranza convencional,Otro Criolla,CRIOLLO,"MULTIPOLYGON (((-102.58961 22.98375, -102.5887..."
1,BitAg-2022-0525-0312-tFiZR-NONKi,AE,Innovacion,RParc-2022-0518-1025-KuiWV-pG8zK,Sitio-2022-0413-1202-gykMQ-coVuZ,AJO,Zacatecas,Calera,OI_2021_2022,OI,RIEGO,4.5,4.5,Ajo,Labranza minima reducida,Otro Criolla,AJO,"MULTIPOLYGON (((-102.65673 22.93083, -102.6555..."


In [147]:
#join two sets
gdf_consolidado_pv22 = same_pro_last_set.append(new_geodf,ignore_index=True)

C:\Users\JMELENDEZ\AppData\Local\Temp\ipykernel_13092\1867640418.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf_consolidado_pv22 = same_pro_last_set.append(new_geodf,ignore_index=True)


In [148]:
gdf_consolidado_pv22.tail(2)

,ID,Tipo de bitacora,Tipo de superficie,ID parcela,ID del sitio,Nombre de la Parcela,Estado,Municipio,Anio Bitacora,Ciclo,Regimen hidrico,Superficie sembrada,Superficie Total,Cultivo,Labranza,Tipo de semilla,Variedad,geometry
451,BitAg-2023-0305-1122-ZDyT7-VCAE4,AE,Innovacion,RParc-2023-0302-0615-PLnHz-wH3oj,Sitio-2023-0302-0608-5lmVG-IYHAy,EL SABINO 3,Zacatecas,Fresnillo,PV_2022,PV,RIEGO,9.9,9.90,Chile,Labranza_convencional,Chile_Hibrida,MIRASOL,"POLYGON ((-102.63599 23.10034, -102.63434 23.1..."
452,BitAg-2023-0327-1157-fWfxN-vhrwK,AI,Innovacion,RParc-2023-0327-1156-otqb5-rYPUE,Sitio-2023-0327-1154-atgXf-8qh36,RANCHO MONTE DE URIBE AI 4,Zacatecas,Fresnillo,PV_2022,PV,RIEGO,1.7,3.91,Chile,None,Chile_Criolla,MIRASOL,"POLYGON ((-102.78218 23.16718, -102.78042 23.1..."


In [149]:
gdf_consolidado_pv22.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 453 entries, 0 to 452
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   ID                    453 non-null    object  
 1   Tipo de bitacora      453 non-null    object  
 2   Tipo de superficie    453 non-null    object  
 3   ID parcela            453 non-null    object  
 4   ID del sitio          453 non-null    object  
 5   Nombre de la Parcela  453 non-null    object  
 6   Estado                453 non-null    object  
 7   Municipio             453 non-null    object  
 8   Anio Bitacora         453 non-null    object  
 9   Ciclo                 453 non-null    object  
 10  Regimen hidrico       453 non-null    object  
 11  Superficie sembrada   453 non-null    float64 
 12  Superficie Total      453 non-null    float64 
 13  Cultivo               453 non-null    object  
 14  Labranza              390 non-null    object  
 15

In [150]:
gdf_consolidado_pv22.drop_duplicates(keep='last',inplace=True)

In [151]:
gdf_consolidado_pv22.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 453 entries, 0 to 452
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   ID                    453 non-null    object  
 1   Tipo de bitacora      453 non-null    object  
 2   Tipo de superficie    453 non-null    object  
 3   ID parcela            453 non-null    object  
 4   ID del sitio          453 non-null    object  
 5   Nombre de la Parcela  453 non-null    object  
 6   Estado                453 non-null    object  
 7   Municipio             453 non-null    object  
 8   Anio Bitacora         453 non-null    object  
 9   Ciclo                 453 non-null    object  
 10  Regimen hidrico       453 non-null    object  
 11  Superficie sembrada   453 non-null    float64 
 12  Superficie Total      453 non-null    float64 
 13  Cultivo               453 non-null    object  
 14  Labranza              390 non-null    object  
 15

In [152]:
gdf_consolidado_pv22.tail(6)

,ID,Tipo de bitacora,Tipo de superficie,ID parcela,ID del sitio,Nombre de la Parcela,Estado,Municipio,Anio Bitacora,Ciclo,Regimen hidrico,Superficie sembrada,Superficie Total,Cultivo,Labranza,Tipo de semilla,Variedad,geometry
447,BitAg-2023-0227-0809-XP4NN-f8cgh,AE,Innovacion,RParc-2022-0820-0636-IlrZB-XLsuL,Sitio-2023-0227-0805-i1wYr-51Fvx,EL PINO,Zacatecas,Fresnillo,PV_2022,PV,RIEGO,8.00,8.90,Maiz,Labranza_convencional,Maiz_Hibrida,H_311,"POLYGON ((-102.81060 23.18878, -102.81060 23.1..."
448,BitAg-2023-0301-0239-BXqoB-QK6Fk,AE,Innovacion,RParc-2023-0227-0812-bZUif-DMJf4,Sitio-2022-0712-1147-cq6m0-Jlcef,EL PINO,Zacatecas,Fresnillo,PV_2022,PV,RIEGO,4.25,4.25,Chile,Labranza_convencional,Chile_Criolla,DE ARBOL,"POLYGON ((-102.81055 23.18884, -102.81043 23.1..."
449,BitAg-2023-0302-1240-19ECL-VHaMM,AE,Innovacion,RParc-2022-0909-0248-dxV4a-UHPs2,Sitio-2022-0909-1216-iJnsj-VL9T7,LLANO BLANCO SUR,Zacatecas,Panuco,PV_2022,PV,RIEGO,1.00,5.00,Frijol,Labranza_convencional,Frijol_Mejorada,FLOR DE MAYO,"POLYGON ((-102.61850 23.01135, -102.61588 23.0..."
450,BitAg-2023-0303-0350-OBTWI-IyvSd,AE,Innovacion,RParc-2023-0302-0614-3J0PF-bfmOp,Sitio-2023-0302-0559-92dkE-y8HuK,EL SABINO 2,Zacatecas,Fresnillo,PV_2022,PV,RIEGO,3.00,9.80,Maiz,Labranza_convencional,Maiz_Hibrida,H_319,"POLYGON ((-102.64119 23.10376, -102.63601 23.1..."
451,BitAg-2023-0305-1122-ZDyT7-VCAE4,AE,Innovacion,RParc-2023-0302-0615-PLnHz-wH3oj,Sitio-2023-0302-0608-5lmVG-IYHAy,EL SABINO 3,Zacatecas,Fresnillo,PV_2022,PV,RIEGO,9.90,9.90,Chile,Labranza_convencional,Chile_Hibrida,MIRASOL,"POLYGON ((-102.63599 23.10034, -102.63434 23.1..."
452,BitAg-2023-0327-1157-fWfxN-vhrwK,AI,Innovacion,RParc-2023-0327-1156-otqb5-rYPUE,Sitio-2023-0327-1154-atgXf-8qh36,RANCHO MONTE DE URIBE AI 4,Zacatecas,Fresnillo,PV_2022,PV,RIEGO,1.70,3.91,Chile,None,Chile_Criolla,MIRASOL,"POLYGON ((-102.78218 23.16718, -102.78042 23.1..."


In [154]:
salida=PATH + 'gdf_consolidado_pv22.geojson'

In [155]:
gdf_consolidado_pv22.to_file(salida,crs='CRS84', driver='GeoJSON')